In [1]:
!nvidia-smi

Thu Apr 24 16:51:51 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.30                 Driver Version: 546.30       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   43C    P5               3W /  75W |    630MiB /  4096MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda') # torch.cuda.device('cuda:1') 访问第一个GPU


(device(type='cpu'), <torch.cuda.device at 0x22e65b0f6d0>)

In [3]:
# 定义两个函数来测试是否存在GPU如果没有则用CPU
def try_gpu(i=0): #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
    
def try_all_gpus(): #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
        for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]
    
try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [4]:
# 查询张量坐在的设备
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
# 创建张量时，放在GPU上
X = torch.ones(2, 3, device=try_gpu())
X

# 也可以在第二个GPU上创建张量
# Y = torch.rand(2, 3, device=try_gpu(1))

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [9]:
# 如果你有多个GPU，可以将一个GPU上的值copy到另一个GPU上
#Z = X.cuda(1) # 将X的值从GPU0 copy到GPU1
# 数值在一个GPU上才能做计算

In [11]:
# 在GPU上做神经网络
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())  # net.to将神经网络挪动到0号GPU上 ,等于将网络的参数在0号GPU上copy一份

net(X)  # x也在0号GPU上


tensor([[0.3612],
        [0.3612]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [12]:
net[0].weight.data.device

device(type='cuda', index=0)